# Building virtual rasters from STAC

Please check out my blog post associated to this code if you would like some expansion on the code shown below:
http://www.acgeospatial.co.uk/building-virtual-rasters-from-stac

In [1]:
from satsearch import Search
import geopandas as gpd
gdf = gpd.read_file('map.geojson')
bounds = gdf.bounds
boundary = bounds.values.tolist()
search = Search(bbox=boundary[0], datetime='2020-05-01/2020-07-30', 
                collections=['sentinel-s2-l2a-cogs'], url='https://earth-search.aws.element84.com/v0',
               query={'eo:cloud_cover': {'lt': 5}})
print('bbox search: %s items' % search.found())

bbox search: 37 items


In [2]:
items = search.items()
url_dict = {}
for item in items:
    if item.id[4:9] == "30UXB":
        file_url = item.asset('visual')['href']
        fileout = item.id+'.vrt'
        url_dict[fileout] = file_url

In [3]:
print(len(url_dict))
print(url_dict)

9
{'S2B_30UXB_20200730_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/7/S2B_30UXB_20200730_0_L2A/TCI.tif', 'S2A_30UXB_20200625_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/6/S2A_30UXB_20200625_0_L2A/TCI.tif', 'S2B_30UXB_20200623_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/6/S2B_30UXB_20200623_0_L2A/TCI.tif', 'S2B_30UXB_20200531_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/5/S2B_30UXB_20200531_0_L2A/TCI.tif', 'S2A_30UXB_20200529_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/5/S2A_30UXB_20200529_0_L2A/TCI.tif', 'S2A_30UXB_20200526_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2020/5/S2A_30UXB_20200526_0_L2A/TCI.tif', 'S2B_30UXB_20200521_0_L2A.vrt': 'https://sentinel-cogs.s3.us-west-2.amazonaws.c

In [4]:
import os
output_dir = 'D:\my_project\sat_links_TCI'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
os.chdir(output_dir)

In [5]:
import subprocess
for _, (key, value) in enumerate(url_dict.items()):
    subprocess.call('gdalbuildvrt ' + key + ' ' + value)